In [ ]:
start_date = datetime.datetime(2023, 7, 1)
end_date = datetime.datetime(2024, 11, 1)

start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

In [ ]:
import yfinance as yf

# Scarica i dati per l'indice S&P 500
sp500 = yf.Ticker("^GSPC")

# Dati storici dell'S&P 500 (per esempio, degli ultimi 5 anni)
#data = sp500.history(period="max", interval="1m")
data = yf.download("^GSPC", start=start_date_str, end=end_date_str, interval="1mo")

print(data)

In [ ]:
import pandas as pd
import datetime as datetime

url_raw = "https://yfiua.github.io/index-constituents/$YYYY/$MM/constituents-sp500.csv"

# calcolo i costituentu dell'indice S&P 500 mese per mese a partire dal 2023/07 fino al 2024/10

stock_dict = {}

for date in pd.date_range(start_date, end_date, freq='MS'):
    print(date)
    url = url_raw.replace("$YYYY", str(date.year)).replace("$MM", str(date.month).zfill(2))
    print(url)
    # scarico il file CSV
    constituents = pd.read_csv(url)
    # aggiungo i dati storici
    print(constituents)
    for symbol in constituents['Symbol']:
        print(symbol)
        # Scarica i dati per il simbolo e salvali in un file CSV
        symbol_data = yf.download(symbol, start=start_date_str, end=end_date_str, interval="1mo")
        #print(symbol_data)
        if symbol not in stock_dict:
            stock_dict[symbol] = symbol_data
            #symbol_data.to_csv("stocks/" + symbol + ".csv")
        

    

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 1. URL della pagina Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"  # Sostituisci con l'URL della pagina di interesse

# 2. Richiesta HTTP alla pagina
response = requests.get(url)
if response.status_code == 200:
    # 3. Parsing del contenuto HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # 4. Trovare la tabella specifica
    # Cerca la tabella basandoti su una classe CSS o un indice se ci sono più tabelle
    table = soup.find("table", {"id": "constituents"})  # 'wikitable' è comune nelle tabelle di Wikipedia

    # 5. Estrazione dei dati dalla tabella
    rows = []
    headers = [header.text.strip() for header in table.find_all("th")]  # Estrae le intestazioni
    for row in table.find_all("tr")[1:]:  # Salta la riga delle intestazioni
        cells = row.find_all(["td", "th"])  # Prende sia td che th, nel caso di celle di intestazione
        rows.append([cell.text.strip() for cell in cells])

    # 6. Converti i dati in un DataFrame di Pandas (opzionale ma utile)
    df = pd.DataFrame(rows, columns=headers)
    #print(df)
    # 7. Salva i dati in un file CSV
    df.to_csv("sp500_companies.csv", index=False)

    table = soup.find("table", {"id": "changes"})  # 'wikitable' è comune nelle tabelle di Wikipedia

    # 5. Estrazione dei dati dalla tabella
    rows = []
    headers = ['Date', 'Ticker_added', 'Security_added', 'Ticker_removed', 'Security_removed', 'Reason']  # Estrae le intestazioni
    for row in table.find_all("tr")[2:]:  # Salta la riga delle intestazioni
        cells = row.find_all(["td", "th"])  # Prende sia td che th, nel caso di celle di intestazione
        rows.append([cell.text.strip() for cell in cells])

    # 6. Converti i dati in un DataFrame di Pandas (opzionale ma utile)
    df2 = pd.DataFrame(rows, columns=headers)

    # trasformo la data in un formato migliore
    df2['Date'] = pd.to_datetime(df2['Date'])
    #print(df2)
    # 7. Salva i dati in un file CSV
    df2.to_csv("sp500_companies_changes.csv", index=False)


else:
    print(f"Errore nella richiesta: {response.status_code}")


In [26]:
import datetime as datetime

# Creo un data frame con la composizione dell'indice S&P 500 nel tempo a partire dai due csv

# carico i due file csv
df_companies = pd.read_csv("sp500_companies.csv")
df_changes = pd.read_csv("sp500_companies_changes.csv")

# creo un dizionario con le date e i ticker delle aziende
date_ticker_dict = {}
today = datetime.datetime.now().strftime('%Y-%m-%d')
# aggiungo le aziende presenti nel file df_companies con la data di oggi
current_composition = []
for index, row in df_companies.iterrows():
    current_composition.append(row['Symbol'])

c = 0
# aggiorno il dizionario con le aziende aggiunte e rimosse
print(current_composition)
for index, row in df_changes.iterrows():
    date_ticker_dict[row['Date']] = current_composition.copy()
    # prendo la composizione attuale e aggiorno in base ai cambiamenti
    if row['Ticker_removed'] is not None and pd.isna(row['Ticker_removed']) == False:
        #current_composition = [x for x in current_composition if x != row['Ticker_removed']]
        current_composition.append(row['Ticker_removed'])
    if row['Ticker_added'] is not None and pd.isna(row['Ticker_added']) == False:
        if row['Ticker_added'] not in current_composition:
            print(row['Ticker_added'])
            c += 1
        else:
            current_composition.remove(row['Ticker_added'])
    current_composition = current_composition.copy()

print(c)
# creo un data frame con la composizione dell'indice S&P 500 nel tempo
df_composition = pd.DataFrame(date_ticker_dict.items(), columns=['Date', 'Composition'])

#scrivo il file csv
df_composition.to_csv("sp500_composition.csv", index=False)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMTM', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', '